In [72]:
#大目的：选择特征，训练模型并测试
#画出监测图

In [4]:
import pandas as pd
import xgboost as xgb
import numpy as np
from pandas import DataFrame

In [5]:
#获取数据
def read_data (data_path):
    data = pd.read_excel(data_path)
    #print (data.head(5))
    return data

In [10]:
#1、对数据进行One_hot编码；2、修改标签列索引；3、将数据转化为DMatrix结构
def data_preprocess (data):
    #执行one_hot编码
    data = pd.get_dummies(data)
    #print (data.head(5))
    
    #将最后一列变为标签列
    #print (data.columns)
    data.rename(columns = {"车流量":"label"}, inplace = True)
    #print (data.head(5))
    #print (len(data))
    
    #将数据拆分为训练集和测试集
    bool_list = np.random.rand(len(data)) < 0.8
    train_data = data[bool_list]
    neg_bool_list = (bool_list == False)
    test_data = data[neg_bool_list] 
    
    #将数据转换成DMatrix格式，变为符合xgboost模型的数据
    data_col_num = train_data.shape[1]
    xgb_train = xgb.DMatrix(train_data.iloc[:,:data_col_num], label = train_data.label)
    xgb_test = xgb.DMatrix(test_data.iloc[:,:data_col_num], label = test_data.label)
    return xgb_train, xgb_test

In [11]:
#模型训练
def train_model(xgb_train, xgb_test):
    #将train_data，test_data分别转换为DMatrix结构
    #data_col_num = train_data.shape[1]
    #xgb_train = xgb.DMatrix(train_data.iloc[:,:data_col_num], label=train_data.label)
    #xgb_test = xgb.DMatrix(test_data.iloc[:,:data_col_num], label=test_data.label)
    
    #定义模型参数
    params = {"objective":"reg:linear", "booster":"gbtree", "eta":0.1, "min_child_weight":1, "max_depth":5}
    num_round = 30
    watchlist = [(xgb_train, "train"), (xgb_test, "test")]
    model = xgb.train(params, xgb_train, num_round, watchlist)
    model.save_model("E:\\F_Paper\\Xgboost_test\\model.xgb")
    print("模型训练完成，并存储成功")

In [12]:
if __name__ == '__main__':
    data_path = 'E://F_Paper/one_hot feature data preprocess test.xlsx'
    data = read_data(data_path)
    xgb_train, xgb_test = data_preprocess(data)
    #params = {"objective":"reg:linear", "booster":"gbtree", "eta":0.1, "min_child_weight":1, "max_depth":5}
    #num_round = 30
    #watchlist = [(xgb_train, "train"), (xgb_test, "test")]
    #model = xgb.train(params, xgb_train, num_round, watchlist)
    #model.save_model("E:\\F_Paper\\Xgboost_test\\model.xgb")
    #print("模型训练完成，并存储成功")
    train_model(xgb_train, xgb_test)

   res颗粒物一小时平均浓度  res空气质量指数  res颗粒物日滑动平均浓度  air空气质量污染指数  降雨量   风速  最低温度    湿度  \
0             54         82             46          109    0  1.4  17.0  73.0   
1             73         77             56          173    0  1.4  20.0  73.0   
2             83        113             76          120    0  1.4  16.0  73.0   
3             54         74             54          132    0  1.4  17.0  73.0   
4             74        100             75          191    0  1.8  19.0  75.0   

    车流量  air空气质量_中度污染  air空气质量_轻度污染  air空气指数颜色_橙色  air空气指数颜色_红色  
0  6978             0             1             1             0  
1  4891             1             0             0             1  
2  3791             0             1             1             0  
3  5223             0             1             1             0  
4  2376             1             0             0             1  
[19:25:29] WARNING: d:\build\xgboost\xgboost-1.2.1.git\src\objective\regression_obj.cu:174: reg:linear is now depreca

In [9]:
#模型测试
bst = xgb.Booster()
bst.load_model("E:\\F_Paper\\Xgboost_test\\model.xgb")
pred = bst.predict(xgb_test)
print (pred)

[19:25:10] WARNING: d:\build\xgboost\xgboost-1.2.1.git\src\objective\regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[6231.124 6231.124 4923.322]
